# This is the final code 
#### get the data from dashboard api and make my process


In [1]:
import cv2
import requests
from pyzbar.pyzbar import decode
import pyttsx3


# you should put your own url .
def send_to_api(code):
    url = f"https://802b-197-53-109-195.ngrok-free.app/api/Product/{code}"
    response = requests.get(url)
    return response

def decode_qr_code(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    decoded_objects = decode(gray)
    if decoded_objects:
        return decoded_objects[0].data.decode('utf-8')
    else:
        return None


def speak(text):
    engine = pyttsx3.init()

    # Adjust voice properties as desired
    rate = engine.getProperty('rate')
    engine.setProperty('rate', rate - 45)  # Decrease speaking rate by 35 (adjust as needed)
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[1].id)  # Set voice to second voice (adjust index)

    engine.say(text)
    engine.runAndWait()

def capture_and_process_qr():
    voice_rate_adjusted = False
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            qr_data = decode_qr_code(frame)
            if qr_data:
                # Send QR code data to your API
                response = send_to_api(qr_data)
                if response.status_code == 200:
                    data = response.json()
                    
                    # Extract product information from API response (assuming structure)
                    product_name = data.get('product_name')
                    description = data.get('description')
                    price = data.get('price')
                    currency = data.get('currency')
                    
                    # Construct product information text for TTS
                    product_text = f"Product Name: {product_name}\nDescription: {description}\nPrice: {price} {currency}"
                    
                    # Speak the product information
                    speak(product_text)
                else:
                    print(f"Error: {response.status_code}")

            cv2.imshow('QR Code Scanner', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

capture_and_process_qr()


Error: 404


# Old version 
### read the data from the local pc


In [15]:
import cv2
from pyttsx3 import init
import csv
import cv2
import pandas as pd
from statistics import mode
from pyzbar.pyzbar import decode

# Read the CSV file and specify the data types
df = pd.read_json('Products (1).json')
df= df.drop(['picture_url','brand_id','category_id' ], axis = 1)
# Initialize text-to-speech engine
engine = init()
rate = engine.getProperty('rate')
engine.setProperty('rate', rate -35)  # Decrease the rate by 50 (adjust as needed)
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

# Function to speak out text
def speak(text):
    engine.say(text)
    engine.runAndWait()

# Function to decode QR code from image
def decode_qr_code(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Decode QR code
    decoded_objects = decode(gray)
    
    # If QR code is detected, return decoded data
    if decoded_objects:
        return decoded_objects[0].data.decode('utf-8')
    else:
        return None

# Capture image from webcam
def capture_image():
    scanned_products = set()
    scanned_products_file = 'scanned_products.csv' 
    detected_products = []
    selected_products = []
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            cv2.imshow('QR Code Scanner', frame)
            qr_data = decode_qr_code(frame)
            if qr_data:
                if qr_data not in selected_products:                    
                    if len(detected_products) == 10:
                        product_id = mode(detected_products)
                        
                        # Check if the condition exists in the DataFrame
                        
                        result = df[df['code'] == product_id]
                        if not result.empty:
                            product_info = result[['product_name', 'description', 'price', 'currency']]
                            # Convert product_info to text and speak it out
                            product_text = product_info.to_string(index=False, header=False)
                            speak(product_text)
                        selected_products.append(mode(detected_products))
                        detected_products = []
                        try:
                            with open(scanned_products_file, 'a', newline='') as csvfile:
                                writer = csv.writer(csvfile)
                                writer.writerow(product_text.split('\n'))  # Write each line separately
                        except IOError as e:
                            print(f"Error saving scanned product data: {e}")

                        # Add scanned product to the set
                        scanned_products.add(qr_data)
                    else:
                        detected_products.append(qr_data)
                else:
                    continue
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

# Call function
capture_image()


